In [ ]:
# 필요한 라이브러리 임포트
import os
import time
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from tqdm import tqdm
import re

print("라이브러리 임포트 완료")


In [ ]:
# .env에서 API 키 로드
load_dotenv()
SERVICE_KEY = os.getenv("BUILDING_HUB_API_KEY")
if not SERVICE_KEY:
    raise ValueError("환경변수 BUILDING_HUB_API_KEY가 설정되어 있지 않습니다.")

print("API 키 로드 완료")

# 데이터 파일 로드 - new_주소 컬럼이 있는 파일 찾기
def load_data_with_new_address():
    """new_주소 컬럼이 있는 데이터 파일을 로드하는 함수"""
    
    # 가능한 파일들 확인
    possible_files = [
        "../data/batch_stage_matching_result_final.xlsx",
        "../data/text_matching_result_ver3_요약 정리_20250415.xlsx", 
        "../data/대학교_매칭_결과_개선_ver3.xlsx"
    ]
    
    for file_path in possible_files:
        try:
            if os.path.exists(file_path):
                print(f"파일 로드 시도: {file_path}")
                df = pd.read_excel(file_path)
                
                # new_주소 컬럼이 있는지 확인
                if 'new_주소' in df.columns:
                    print(f"✓ new_주소 컬럼 발견! 파일: {file_path}")
                    print(f"  데이터 크기: {df.shape}")
                    print(f"  new_주소 결측치: {df['new_주소'].isna().sum()}개")
                    return df
                else:
                    print(f"  new_주소 컬럼 없음")
            else:
                print(f"파일 없음: {file_path}")
        except Exception as e:
            print(f"파일 로드 오류: {file_path} - {e}")
    
    return None

# 데이터 로드
df = load_data_with_new_address()

if df is not None:
    print(f"\n=== 데이터 로드 성공 ===")
    print(f"전체 데이터: {len(df)}건")
    print(f"new_주소 샘플:")
    print(df['new_주소'].dropna().head(3).tolist())
else:
    print("new_주소 컬럼이 있는 데이터 파일을 찾을 수 없습니다.")

In [ ]:
# new_주소를 API 요청 파라미터로 변환하는 함수
def parse_address_to_api_params(address):
    """
    new_주소를 API 요청에 필요한 파라미터로 변환
    
    Parameters:
    -----------
    address : str
        예: "서울특별시 강남구 역삼동 100-5번지"
    
    Returns:
    --------
    dict : API 파라미터
    """
    if pd.isna(address) or not isinstance(address, str):
        return None
    
    # 기본값 설정
    result = {
        'SIGUNGU_CD': '',
        'BJDONG_CD': '',
        'PLAT_GB_CD': '0',  # 기본값: 일반(0)
        'BUN': '',
        'JI': ''
    }
    
    try:
        # 주소에서 번지 추출
        bun_ji_pattern = r'(\d+)(?:-(\d+))?번지'
        bun_ji_match = re.search(bun_ji_pattern, address)
        
        if bun_ji_match:
            bun = bun_ji_match.group(1)
            ji = bun_ji_match.group(2) if bun_ji_match.group(2) else '0'
            result['BUN'] = bun
            result['JI'] = ji
        
        # '산' 여부 확인 (대지구분코드)
        if '산' in address:
            result['PLAT_GB_CD'] = '1'  # 산: 1
        
        print(f"주소 파싱 결과: {address[:50]}... -> BUN:{result['BUN']}, JI:{result['JI']}, 산여부:{result['PLAT_GB_CD']}")
        
    except Exception as e:
        print(f"주소 파싱 오류: {address} - {e}")
        return None
    
    return result

# 데이터에 변환 적용
if df is not None:
    print("\n=== new_주소 -> API 파라미터 변환 ===")
    
    # new_주소가 있는 데이터만 필터링
    df_with_address = df[df['new_주소'].notna()].copy()
    print(f"new_주소가 있는 데이터: {len(df_with_address)}건")
    
    # 샘플 데이터로 테스트
    sample_addresses = df_with_address['new_주소'].head(5)
    print(f"\n샘플 주소 파싱 테스트:")
    
    for i, addr in enumerate(sample_addresses):
        print(f"\n{i+1}. 원본: {addr}")
        parsed = parse_address_to_api_params(addr)
        if parsed:
            print(f"   변환: BUN={parsed['BUN']}, JI={parsed['JI']}, 산={parsed['PLAT_GB_CD']}")
        else:
            print(f"   변환 실패")
            
else:
    print("데이터가 로드되지 않았습니다.")


In [ ]:
# 기본개요 API 호출 함수 (수정된 버전)
def fetch_basic_outline_data(row):
    """
    기본개요 데이터를 API로 조회하는 함수
    
    Parameters:
    -----------
    row : pandas.Series
        SIGUNGU_CD, BJDONG_CD, PLAT_GB_CD, BUN, JI 컬럼이 있는 행
    
    Returns:
    --------
    list : API 응답 결과 리스트
    """
    params = {
        "serviceKey": SERVICE_KEY,
        "sigunguCd": row.get("SIGUNGU_CD", ""),
        "bjdongCd": row.get("BJDONG_CD", ""),
        "platGbCd": row.get("PLAT_GB_CD", "0"),
        "bun": str(row.get("BUN", "")).zfill(4),      # "12" → "0012"
        "ji": str(row.get("JI", "0")).zfill(4),       # "0"  → "0000"
        "numOfRows": "100",
        "pageNo": "1",
        "_type": "json",
    }
    
    try:
        response = requests.get(
            "https://apis.data.go.kr/1613000/BldRgstHubService/getBrBasisOulnInfo",
            params=params
        )
        response.raise_for_status()
        
        data = response.json()
        items = (data
                   .get("response", {})
                   .get("body", {})
                   .get("items", {})
                   .get("item"))
        
        # API가 dict 하나만 줄 수도, list를 줄 수도 있으므로 통일
        if not items:
            return []
        return items if isinstance(items, list) else [items]
        
    except Exception as e:
        print(f"API 호출 오류: {e}")
        return []

# 실제 데이터 수집 실행 (예시)
def collect_basic_outline_data():
    """기본개요 데이터 일괄 수집"""
    
    if df is None:
        print("데이터가 로드되지 않았습니다.")
        return
    
    # new_주소가 있는 데이터만 필터링
    df_with_address = df[df['new_주소'].notna()].copy()
    
    # 각 행에 대해 API 파라미터 생성
    api_params_list = []
    for idx, row in df_with_address.iterrows():
        parsed = parse_address_to_api_params(row['new_주소'])
        if parsed and parsed['BUN']:  # BUN이 있는 경우만
            parsed['original_index'] = idx
            api_params_list.append(parsed)
    
    print(f"\nAPI 호출 대상: {len(api_params_list)}건")
    
    if len(api_params_list) == 0:
        print("API 호출할 데이터가 없습니다.")
        return
    
    # 사용자 확인
    proceed = input(f"{len(api_params_list)}건의 데이터에 대해 API를 호출하시겠습니까? (y/n): ")
    
    if proceed.lower() != 'y':
        print("API 호출을 취소했습니다.")
        return
    
    # API 호출 실행
    all_records = []
    failed_count = 0
    
    print(f"\n기본개요 데이터 수집 시작...")
    
    for i, params in enumerate(tqdm(api_params_list, desc="API 호출 중")):
        try:
            records = fetch_basic_outline_data(params)
            if records:
                # 원본 인덱스 정보 추가
                for record in records:
                    record['original_index'] = params['original_index']
                all_records.extend(records)
            else:
                failed_count += 1
            
            # API 호출 간격 조절
            time.sleep(0.1)
            
        except Exception as e:
            print(f"오류 발생 (인덱스 {params['original_index']}): {e}")
            failed_count += 1
    
    # 결과 저장
    if all_records:
        df_result = pd.DataFrame(all_records)
        filename = "기본개요_건축HUB_new주소변환.csv"
        df_result.to_csv(filename, index=False, encoding="utf-8-sig")
        
        print(f"\n=== 수집 완료 ===")
        print(f"총 {len(df_result)}건 수집됨")
        print(f"실패: {failed_count}건")
        print(f"저장 파일: {filename}")
        
        # 결과 미리보기
        print(f"\n결과 미리보기:")
        print(df_result.head())
        
    else:
        print("수집된 데이터가 없습니다.")

# 수집 함수 정의만 하고 실행은 하지 않음
print("기본개요 데이터 수집 함수가 준비되었습니다.")
print("실행하려면: collect_basic_outline_data() 를 호출하세요.")


In [ ]:
# 해결방법 1: 기존 건축HUB 데이터에서 주소-코드 매핑 생성
def create_address_code_mapping():
    """기존 수집된 건축HUB 데이터에서 주소 → 코드 매핑 테이블 생성"""
    
    # 기존 수집된 파일들 확인
    hub_files = [
        "../data/표제부_건축HUB.csv",
        "../data/기본개요_건축HUB.csv", 
        "../data/총괄표제부_건축HUB.csv"
    ]
    
    mapping_df = None
    
    for file_path in hub_files:
        if os.path.exists(file_path):
            print(f"매핑 데이터 로드: {file_path}")
            try:
                temp_df = pd.read_csv(file_path, encoding='utf-8-sig')
                
                # 필요한 컬럼이 있는지 확인
                required_cols = ['platPlc', 'sigunguCd', 'bjdongCd']
                if all(col in temp_df.columns for col in required_cols):
                    # 매핑에 필요한 컬럼만 추출
                    temp_mapping = temp_df[required_cols].drop_duplicates()
                    
                    if mapping_df is None:
                        mapping_df = temp_mapping
                    else:
                        mapping_df = pd.concat([mapping_df, temp_mapping]).drop_duplicates()
                        
                    print(f"  → {len(temp_mapping)}개 주소-코드 매핑 추가")
                else:
                    print(f"  → 필요한 컬럼 없음: {temp_df.columns.tolist()}"[:100])
                    
            except Exception as e:
                print(f"  → 파일 로드 오류: {e}")
        else:
            print(f"파일 없음: {file_path}")
    
    if mapping_df is not None:
        print(f"\n총 {len(mapping_df)}개의 고유 주소-코드 매핑 생성됨")
        return mapping_df
    else:
        print("주소-코드 매핑 데이터를 찾을 수 없습니다.")
        return None

# 매핑 테이블 생성
address_mapping = create_address_code_mapping()

if address_mapping is not None:
    print("\n=== 주소-코드 매핑 샘플 ===")
    print(address_mapping.head())
    
    # 시군구별 분포 확인
    print(f"\n시군구코드 분포 (상위 10개):")
    print(address_mapping['sigunguCd'].value_counts().head(10))


In [ ]:
# 해결방법 2: 스마트한 주소 매칭 함수
def smart_address_matching(target_address, mapping_df):
    """
    유사도 기반 주소 매칭
    
    Parameters:
    -----------
    target_address : str
        매칭할 대상 주소
    mapping_df : DataFrame
        기존 주소-코드 매핑 테이블
        
    Returns:
    --------
    dict : 매칭된 시군구코드, 법정동코드
    """
    if mapping_df is None or len(mapping_df) == 0:
        return None
    
    # 대상 주소 정규화
    target_clean = re.sub(r'[^\w\s]', '', target_address)
    target_clean = re.sub(r'\s+', ' ', target_clean).strip()
    
    best_match = None
    best_score = 0
    
    # 정확한 매칭 시도 (부분 문자열)
    for idx, row in mapping_df.iterrows():
        map_address = str(row['platPlc'])
        map_clean = re.sub(r'[^\w\s]', '', map_address)
        map_clean = re.sub(r'\s+', ' ', map_clean).strip()
        
        # 1. 완전 일치
        if target_clean == map_clean:
            return {
                'sigunguCd': row['sigunguCd'],
                'bjdongCd': row['bjdongCd'],
                'matched_address': map_address,
                'match_type': '완전일치'
            }
        
        # 2. 포함 관계 (대상 주소가 매핑 주소에 포함)
        if target_clean in map_clean or map_clean in target_clean:
            # 길이 유사도 계산
            score = min(len(target_clean), len(map_clean)) / max(len(target_clean), len(map_clean))
            if score > best_score:
                best_score = score
                best_match = {
                    'sigunguCd': row['sigunguCd'],
                    'bjdongCd': row['bjdongCd'],
                    'matched_address': map_address,
                    'match_type': f'포함매칭({score:.2f})'
                }
    
    # 3. 토큰 기반 매칭
    if best_match is None:
        target_tokens = set(target_clean.split())
        
        for idx, row in mapping_df.iterrows():
            map_address = str(row['platPlc'])
            map_clean = re.sub(r'[^\w\s]', '', map_address)
            map_tokens = set(map_clean.split())
            
            # 공통 토큰 비율
            if len(target_tokens) > 0 and len(map_tokens) > 0:
                common_tokens = target_tokens.intersection(map_tokens)
                score = len(common_tokens) / max(len(target_tokens), len(map_tokens))
                
                if score > 0.5 and score > best_score:  # 50% 이상 매칭
                    best_score = score
                    best_match = {
                        'sigunguCd': row['sigunguCd'],
                        'bjdongCd': row['bjdongCd'],
                        'matched_address': map_address,
                        'match_type': f'토큰매칭({score:.2f})'
                    }
    
    return best_match

# 테스트해보기
if address_mapping is not None and df is not None:
    print("\n=== 스마트 주소 매칭 테스트 ===")
    
    # new_주소가 있는 데이터 샘플
    test_addresses = df[df['new_주소'].notna()]['new_주소'].head(5)
    
    for i, addr in enumerate(test_addresses):
        print(f"\n{i+1}. 대상 주소: {addr}")
        
        # 스마트 매칭 시도
        match_result = smart_address_matching(addr, address_mapping)
        
        if match_result:
            print(f"   ✓ 매칭 성공!")
            print(f"     시군구코드: {match_result['sigunguCd']}")
            print(f"     법정동코드: {match_result['bjdongCd']}")
            print(f"     매칭방식: {match_result['match_type']}")
            print(f"     매칭된 주소: {match_result['matched_address']}")
        else:
            print(f"   ✗ 매칭 실패")

else:
    print("주소 매핑 데이터 또는 대상 데이터가 없습니다.")


In [ ]:
# 개선된 주소 파싱 및 API 호출 함수
def parse_address_to_api_params_v2(address, mapping_df=None):
    """
    new_주소를 API 요청에 필요한 파라미터로 변환 (개선된 버전)
    
    Parameters:
    -----------
    address : str
        예: "강원특별자치도 정선군 북평면 남평리 66-3"
    mapping_df : DataFrame
        주소-코드 매핑 테이블
    
    Returns:
    --------
    dict : API 파라미터
    """
    if pd.isna(address) or not isinstance(address, str):
        return None
    
    # 기본값 설정
    result = {
        'SIGUNGU_CD': '',
        'BJDONG_CD': '',
        'PLAT_GB_CD': '0',  # 기본값: 일반(0)
        'BUN': '',
        'JI': '',
        'original_address': address,
        'match_method': '직접파싱'
    }
    
    try:
        # 1. 스마트 매칭 시도 (매핑 데이터가 있는 경우)
        if mapping_df is not None:
            match_result = smart_address_matching(address, mapping_df)
            if match_result:
                result['SIGUNGU_CD'] = match_result['sigunguCd']
                result['BJDONG_CD'] = match_result['bjdongCd']
                result['match_method'] = f"스마트매칭({match_result['match_type']})"
                result['matched_address'] = match_result['matched_address']
        
        # 2. 번지 추출
        bun_ji_pattern = r'(\d+)(?:-(\d+))?번지'
        bun_ji_match = re.search(bun_ji_pattern, address)
        
        if bun_ji_match:
            bun = bun_ji_match.group(1)
            ji = bun_ji_match.group(2) if bun_ji_match.group(2) else '0'
            result['BUN'] = bun
            result['JI'] = ji
        
        # 3. '산' 여부 확인 (대지구분코드)
        if '산' in address:
            result['PLAT_GB_CD'] = '1'  # 산: 1
        
        # 4. 매핑 데이터가 없는 경우 기본 파싱 (제한적)
        if not result['SIGUNGU_CD'] and not result['BJDONG_CD']:
            result['match_method'] = '매핑실패'
            # 여기서는 코드를 얻을 수 없으므로 빈 값 유지
        
    except Exception as e:
        print(f"주소 파싱 오류: {address} - {e}")
        return None
    
    return result

# 개선된 데이터 수집 함수
def collect_basic_outline_data_v2():
    """기본개요 데이터 일괄 수집 (개선된 버전)"""
    
    if df is None:
        print("데이터가 로드되지 않았습니다.")
        return
    
    # new_주소가 있는 데이터만 필터링
    df_with_address = df[df['new_주소'].notna()].copy()
    print(f"new_주소가 있는 데이터: {len(df_with_address)}건")
    
    # 각 행에 대해 API 파라미터 생성
    api_params_list = []
    parsing_stats = {'성공': 0, '실패': 0, '매핑없음': 0}
    
    print("\\n주소 파싱 중...")
    for idx, row in tqdm(df_with_address.iterrows(), total=len(df_with_address), desc="주소 파싱"):
        parsed = parse_address_to_api_params_v2(row['new_주소'], address_mapping)
        
        if parsed:
            parsed['original_index'] = idx
            
            # API 호출 가능한지 확인 (시군구코드, 법정동코드, 번지가 모두 있어야 함)
            if parsed['SIGUNGU_CD'] and parsed['BJDONG_CD'] and parsed['BUN']:
                api_params_list.append(parsed)
                parsing_stats['성공'] += 1
            elif not parsed['SIGUNGU_CD'] or not parsed['BJDONG_CD']:
                parsing_stats['매핑없음'] += 1
            else:
                parsing_stats['실패'] += 1
        else:
            parsing_stats['실패'] += 1
    
    # 파싱 결과 요약
    print(f"\\n=== 주소 파싱 결과 ===")
    print(f"API 호출 가능: {parsing_stats['성공']}건")
    print(f"매핑 실패: {parsing_stats['매핑없음']}건")
    print(f"파싱 실패: {parsing_stats['실패']}건")
    
    if parsing_stats['성공'] == 0:
        print("API 호출할 데이터가 없습니다.")
        return
    
    # 매칭 방법별 통계
    if api_params_list:
        match_methods = [p['match_method'] for p in api_params_list]
        print(f"\\n매칭 방법별 분포:")
        for method, count in pd.Series(match_methods).value_counts().items():
            print(f"  {method}: {count}건")
    
    # 사용자 확인
    proceed = input(f"\\n{len(api_params_list)}건의 데이터에 대해 API를 호출하시겠습니까? (y/n): ")
    
    if proceed.lower() != 'y':
        print("API 호출을 취소했습니다.")
        return
    
    # API 호출 실행
    all_records = []
    failed_count = 0
    
    print(f"\\n기본개요 데이터 수집 시작...")
    
    for i, params in enumerate(tqdm(api_params_list, desc="API 호출 중")):
        try:
            records = fetch_basic_outline_data(params)
            if records:
                # 원본 정보 추가
                for record in records:
                    record['original_index'] = params['original_index']
                    record['original_address'] = params['original_address']
                    record['match_method'] = params['match_method']
                    if 'matched_address' in params:
                        record['matched_address'] = params['matched_address']
                all_records.extend(records)
            else:
                failed_count += 1
            
            # API 호출 간격 조절
            time.sleep(0.1)
            
        except Exception as e:
            print(f"오류 발생 (인덱스 {params['original_index']}): {e}")
            failed_count += 1
    
    # 결과 저장
    if all_records:
        df_result = pd.DataFrame(all_records)
        filename = "기본개요_건축HUB_스마트매칭.csv"
        df_result.to_csv(filename, index=False, encoding="utf-8-sig")
        
        print(f"\\n=== 수집 완료 ===")
        print(f"총 {len(df_result)}건 수집됨")
        print(f"실패: {failed_count}건")
        print(f"저장 파일: {filename}")
        
        # 결과 미리보기
        print(f"\\n결과 미리보기:")
        display_cols = ['platPlc', 'original_address', 'match_method']
        available_cols = [col for col in display_cols if col in df_result.columns]
        if available_cols:
            print(df_result[available_cols].head())
        else:
            print(df_result.head())
        
        return df_result
        
    else:
        print("수집된 데이터가 없습니다.")
        return None

# 함수 준비 완료
print("\\n=== 개선된 기본개요 데이터 수집 함수 준비 완료 ===")
print("실행하려면: collect_basic_outline_data_v2() 를 호출하세요.")
print("\\n주요 개선사항:")
print("1. 기존 건축HUB 데이터에서 주소-코드 매핑 자동 생성")
print("2. 스마트 주소 매칭 (완전일치 → 포함매칭 → 토큰매칭)")
print("3. 매칭 성공률 및 방법별 통계 제공")
print("4. 원본 주소와 매칭된 주소 정보 보존")
